In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import imageio
from functools import partial
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import os
import itertools
from tqdm.notebook import tqdm
import gc # Importar el Garbage Collector

# Crear un directorio para guardar los GIFs
if not os.path.exists('gifs_cifar10'):
    os.makedirs('gifs_cifar10')
    
# Configurar el dispositivo (GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")


Usando dispositivo: cuda


In [2]:
class LayerMonitor:
    """
    Clase para monitorear el entrenamiento. Almacena datos por capa.
    """
    def __init__(self, model: torch.nn.Module):
        self.model = model
        self.weights = {}
        self.net_values = {}
        self.activations = {}
        self.losses_train = []
        self.losses_val = []

        for name, module in model.named_modules():
            if isinstance(module, nn.Linear): #
                self.weights[name] = [module.weight.clone().detach().cpu().numpy()] # 
            
            elif isinstance(module, nn.ReLU): #
                self.net_values[name] = [] # 
                self.activations[name] = [] # 
                def forward_hook(layer_name, module, input, output):
                    if self.model.training: # 
                        self.net_values[layer_name].append(input[0].clone().detach().cpu().numpy()) # 
                        self.activations[layer_name].append(output.clone().detach().cpu().numpy()) # 
                module.register_forward_hook(partial(forward_hook, name))

    def retrieve_weights_after_optimization_step(self):
        """Recupera los pesos después de un paso de optimización.""" # 
        with torch.no_grad():
            for name, module in self.model.named_modules():
                if isinstance(module, nn.Linear):
                    self.weights[name].append(module.weight.clone().detach().cpu().numpy()) # 

    def receive_losses(self, train_loss: float, val_loss: float):
        """Recibe y almacena las pérdidas.""" # 
        self.losses_train.append(train_loss)
        self.losses_val.append(val_loss)

    def make_movie(self, fig_fun, gif_filename: str, fps: int):
        """Crea la película GIF.""" # 
        frames = []
        num_batches = len(self.losses_train)
        for t in range(num_batches):
            fig = fig_fun(t, self.weights, self.net_values, self.activations, self.losses_train, self.losses_val) # 
            canvas = FigureCanvas(fig)
            canvas.draw()
            frame = np.frombuffer(canvas.buffer_rgba(), dtype='uint8').reshape(canvas.get_width_height()[::-1] + (4,))
            frames.append(frame)
            plt.close(fig)
        if frames:
            imageio.mimsave(gif_filename, frames, fps=fps) # 

def fig_fun(t: int, weights: dict, net_values: dict, activations: dict, train_losses: list, val_losses: list) -> plt.Figure:
    """Genera la figura para cada cuadro del GIF.""" # 
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Training Visualization - Batch Frame {t}', fontsize=16)

    # Panel 1: Curva de Pérdidas 
    axs[0, 0].plot(np.arange(t + 1), train_losses[:t+1], label='Train Loss', color='blue')
    axs[0, 0].plot(np.arange(t + 1), val_losses[:t+1], label='Val Loss', color='red')
    axs[0, 0].set_title('Loss vs Batch')
    axs[0, 0].legend()
    axs[0, 0].grid(True)

    # Panel 2: Distribución de Pesos 
    if t == 0:
        # Lógica para t=0: UN SOLO boxplot consolidado
        all_initial_weights = [wh[0].flatten() for wh in weights.values() if wh]
        if all_initial_weights:
            axs[0, 1].boxplot([np.concatenate(all_initial_weights)], patch_artist=True)
            axs[0, 1].set_title('Initial Weights Distribution (All Layers)')
            axs[0, 1].set_xticklabels(['All Layers'])
    else:
        # Lógica para t > 0: Un boxplot por capa
        weight_data_for_plot = [wh[t].flatten() for wh in weights.values() if t < len(wh)]
        weight_labels = [name for name, wh in weights.items() if t < len(wh)]
        if weight_data_for_plot:
            axs[0, 1].boxplot(weight_data_for_plot, labels=weight_labels, patch_artist=True)
        axs[0, 1].set_title(f'Weights Distribution (Frame {t})')
    
    axs[0, 1].set_ylabel('Weight Value')
    axs[0, 1].grid(True, linestyle='--', alpha=0.7)

    # Panel 3 & 4: Activaciones 
    net_labels = list(net_values.keys())
    net_data = [hist[t].flatten() for hist in net_values.values() if t < len(hist)]
    if net_data:
        axs[1, 0].boxplot(net_data, labels=net_labels, patch_artist=True)
    axs[1, 0].set_title(f'Net Values (Frame {t})')
    
    act_labels = list(activations.keys())
    act_data = [hist[t].flatten() for hist in activations.values() if t < len(hist)]
    if act_data:
        axs[1, 1].boxplot(act_data, labels=act_labels, patch_artist=True)
    axs[1, 1].set_title(f'Activations (Frame {t})')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    return fig


In [3]:
transform_no_std = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)
transform_with_std = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.Lambda(lambda x: x.view(-1))
])

trainset_no_std = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_no_std)
trainset_with_std = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_with_std)
valset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_with_std)

batch_size = 128
# Usar num_workers > 0 para acelerar la carga de datos si no causa problemas
num_workers = 2 if torch.cuda.is_available() else 0
trainloader_no_std = torch.utils.data.DataLoader(trainset_no_std, batch_size=batch_size, shuffle=True, num_workers=num_workers)
trainloader_with_std = torch.utils.data.DataLoader(trainset_with_std, batch_size=batch_size, shuffle=True, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=num_workers)



100%|██████████| 170M/170M [00:29<00:00, 5.70MB/s] 


In [4]:
def init_weights(m, init_method='he'):
    if isinstance(m, nn.Linear):
        if init_method == 'he':
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        elif init_method == 'uniform':
            nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

class FlexibleMLP(nn.Module):
    def __init__(self, input_size=32*32*3, hidden_sizes=[512, 256], output_size=10, 
                 use_batch_norm=False, bn_after_activation=False, dropout_p=0.0):
        super(FlexibleMLP, self).__init__()
        
        self.layers = nn.ModuleList()
        in_size = input_size
        
        for h_size in hidden_sizes:
            self.layers.append(nn.Linear(in_size, h_size))
            if use_batch_norm and not bn_after_activation: # BN antes de ReLU
                self.layers.append(nn.BatchNorm1d(h_size))
            self.layers.append(nn.ReLU())
            if use_batch_norm and bn_after_activation: # BN después de ReLU
                self.layers.append(nn.BatchNorm1d(h_size))
            if dropout_p > 0: # Dropout
                self.layers.append(nn.Dropout(dropout_p))
            in_size = h_size
        self.layers.append(nn.Linear(in_size, output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [6]:
def train_one_configuration(params, loaders, device):
    init_method, standardize, reg_config, bn_config = params
    reg_type, reg_param = reg_config
    bn_option, use_bn = bn_config

    filename = f"init-{init_method}_std-{standardize}_reg-{reg_type}-{reg_param}_bn-{bn_option}.gif"
    filepath = os.path.join('gifs_cifar10', filename)
    print(f"--- Iniciando configuración: {filename} ---")

    trainloader = loaders['with_std'] if standardize else loaders['no_std']
    valloader = loaders['val']

    dropout_p = reg_param if reg_type == 'dropout' else 0.0
    bn_after = bn_option == 'after'
    model = FlexibleMLP(use_batch_norm=use_bn, bn_after_activation=bn_after, dropout_p=dropout_p).to(device)
    model.apply(lambda m: init_weights(m, init_method))
    
    l2_lambda = reg_param if reg_type == 'l2' else 0.0
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l2_lambda)
    criterion = nn.CrossEntropyLoss()

    monitor = LayerMonitor(model)
    
    model.train()
    train_iter = iter(trainloader)
    val_iter = iter(valloader)
    
    # Entrenar por 100 lotes
    for batch_idx in range(100):
        try:
            train_inputs, train_labels = next(train_iter)
        except StopIteration:
            train_iter = iter(trainloader); train_inputs, train_labels = next(train_iter)
        train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(train_inputs)
        train_loss = criterion(outputs, train_labels)
        train_loss.backward()
        optimizer.step()
        
        # **OPTIMIZACIÓN DE MEMORIA**: Solo registrar datos cada 10 lotes
        if batch_idx % 10 == 0:
            monitor.retrieve_weights_after_optimization_step()
            model.eval()
            with torch.no_grad():
                try:
                    val_inputs, val_labels = next(val_iter)
                except StopIteration:
                    val_iter = iter(valloader); val_inputs, val_labels = next(val_iter)
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_loss = criterion(model(val_inputs), val_labels)
            model.train()
            monitor.receive_losses(train_loss.item(), val_loss.item())

    monitor.make_movie(fig_fun, filepath, fps=2)
    print(f"--- GIF guardado en {filepath} ---")

    # Liberación explícita de memoria para las variables más pesadas
    del model, monitor, optimizer, criterion

In [8]:
# Definir el espacio de parámetros para el experimento 
inits = ['he', 'uniform']
standardizations = [True, False]
regularizations = [
    ('none', 0.0), 
    ('l2', 0.01), ('l2', 0.001), ('l2', 0.0001),
    ('dropout', 0.2), ('dropout', 0.5), ('dropout', 0.7)
]
batch_norms = [
    ('none', False), 
    ('before', True), 
    ('after', True)
]

all_params = list(itertools.product(inits, standardizations, regularizations, batch_norms))
print(f"Número total de configuraciones a ejecutar: {len(all_params)}")

dataloaders = {
    'no_std': trainloader_no_std,
    'with_std': trainloader_with_std,
    'val': valloader
}

# Bucle principal
for params in tqdm(all_params, desc="Progreso Total de Experimentos"):
    try:
        train_one_configuration(params, dataloaders, device)
    except Exception as e:
        print(f"!!! ERROR en la configuración {params}: {e} !!!")
        with open("error_log.txt", "a") as f:
            f.write(f"Error en {params}: {e}\n")
    
    # Forzar la recolección de basura y limpiar caché de la GPU después de cada experimento
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\n¡Todos los experimentos han finalizado!")

Número total de configuraciones a ejecutar: 84


Progreso Total de Experimentos:   0%|          | 0/84 [00:00<?, ?it/s]

--- Iniciando configuración: init-he_std-True_reg-none-0.0_bn-none.gif ---


/tmp/ipykernel_96410/30569454.py:87: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[1, 0].boxplot(net_data, labels=net_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:93: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[1, 1].boxplot(act_data, labels=act_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:77: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[0, 1].boxplot(weight_data_for_plot, labels=weight_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:87: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support

--- GIF guardado en gifs_cifar10/init-he_std-True_reg-none-0.0_bn-none.gif ---
--- Iniciando configuración: init-he_std-True_reg-none-0.0_bn-before.gif ---


...

--- GIF guardado en gifs_cifar10/init-uniform_std-False_reg-dropout-0.7_bn-none.gif ---
--- Iniciando configuración: init-uniform_std-False_reg-dropout-0.7_bn-before.gif ---


/tmp/ipykernel_96410/30569454.py:87: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[1, 0].boxplot(net_data, labels=net_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:93: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[1, 1].boxplot(act_data, labels=act_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:77: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[0, 1].boxplot(weight_data_for_plot, labels=weight_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:87: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support

--- GIF guardado en gifs_cifar10/init-uniform_std-False_reg-dropout-0.7_bn-before.gif ---
--- Iniciando configuración: init-uniform_std-False_reg-dropout-0.7_bn-after.gif ---


/tmp/ipykernel_96410/30569454.py:87: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[1, 0].boxplot(net_data, labels=net_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:93: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[1, 1].boxplot(act_data, labels=act_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:77: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[0, 1].boxplot(weight_data_for_plot, labels=weight_labels, patch_artist=True)
/tmp/ipykernel_96410/30569454.py:87: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support

--- GIF guardado en gifs_cifar10/init-uniform_std-False_reg-dropout-0.7_bn-after.gif ---

¡Todos los experimentos han finalizado!
